In [0]:
import csv
from google.colab import drive
drive.mount('/content/drive/')

**Read Dataset**

In [0]:
article_dict={}

with open("/content/drive/My Drive/Colab Notebooks/similarity1.csv",'r') as d1:
    csvreader = csv.reader(d1, delimiter=',')
    next(d1, None)

    for row in csvreader:
      if row[0] not in article_dict:
        article_dict[str(row[0])]={}
        article_dict[str(row[0])]['season']=str(row[3])
        article_dict[str(row[0])]['usage']=str(row[5])
        article_dict[str(row[0])]['mrp']="mrp_"+str(row[8])

In [0]:
ids = []
description = []
with open("/content/drive/My Drive/Colab Notebooks/Live_Styles_Myntra.csv",'r') as d1:
    csvreader = csv.reader(d1, delimiter=',')
    next(d1, None)

    for row in csvreader:
      if row[0] not in article_dict:
        article_dict[str(row[0])]={}
        
#       article_dict[str(row[0])]['gender']=str(row[2])
#       article_dict[str(row[0])]['businessUnit']=str(row[5])
#       article_dict[str(row[0])]['articleType']=str(row[1])
      article_dict[str(row[0])]['styleName']=str(row[4])
      article_dict[str(row[0])]['description']=str(row[6])
      
      sentence = ""
      if 'season' in article_dict[str(row[0])]:
        sentence = article_dict[str(row[0])]['season']+", "+article_dict[str(row[0])]['usage']+", "+article_dict[str(row[0])]['mrp']+", "+article_dict[str(row[0])]['styleName']+", "+article_dict[str(row[0])]['description']
      else:
        sentence = article_dict[str(row[0])]['styleName']+", "+article_dict[str(row[0])]['description']
      if sentence != "":
        ids.append(str("Myntra_"+row[0]))
        description.append(sentence)


**Generate Training Data**

In [0]:
all_ids=[]
with open ("/content/drive/My Drive/Colab Notebooks/All Results.csv",'r') as f:
    csvreader=csv.reader(f, delimiter=',')
    next(f,None)
    for row in csvreader:
      all_ids.append(row[0])
all_ids=all_ids[:12000]

In [0]:
from gensim.models import FastText
from gensim.models import Word2Vec
import sys
import csv
import nltk
from nltk.corpus import stopwords
import re
import numpy
import pickle
import multiprocessing
import gensim
from gensim.models import FastText
import numpy as np

# read word vectors generated and saved before for all descriotions in input dataset
model = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/My Drive/Colab Notebooks/BERT_Embeddings_for_shorter_description.txt")
word_vectors = model.wv

In [0]:
clk_dict={}
for i in all_ids:
  clk_dict[i]=[]
  
    
with open ("/content/drive/My Drive/Colab Notebooks/All Results.csv",'r') as f:
    csvreader=csv.reader(f, delimiter=',')
    next(f,None)
    for row in csvreader:
      if row[0] in clk_dict:
        clk_dict[row[0]]=row[8].split(",")

In [0]:
# last 25 articles using untrained BERT
with open("/content/drive/My Drive/Colab Notebooks/BERT_untrained_last_25_train.csv",'w') as o_file: 
  for w in all_ids:
    if "Myntra_"+w in model.wv:
      l = model.most_similar("Myntra_"+w,topn=17000)
      i = ids.index("Myntra_"+w)
      article=description[i]
      query_article= article.split(",")[0]
      query_gender = article.split(",")[1]
      count=0
      string=""
      for item in l[16950:]:
        if item[0] in clk_dict[w]:
          continue
        i = ids.index(item[0])
        article=description[i]
        result_article = article.split(",")[0]
        result_gender = article.split(",")[1]
        if(query_article.strip()!=result_article.strip() or  query_gender.strip()!=result_gender.strip()):
          continue
        a=item[0]
        string +=str(a.split("_")[1])+","
        count+=1
        if count==25:
          break
      #print(string)
      o_file.write("%s\t%s\n" % ("Myntra_"+w, string))

In [0]:
last_25={}
with open("/content/drive/My Drive/Colab Notebooks/BERT_untrained_last_25_train.csv",'r') as d1:
    csvreader = csv.reader(d1, delimiter='\t')
   # next(d1, None)

    for row in csvreader:
      #print(row)
      id1=row[0].split("_")[1]
      rest=row[1][:-1].split(",")
      last_25[id1]=rest

In [0]:
count=0
with open("/content/drive/My Drive/Colab Notebooks/dataset/train.tsv",'w') as o_file: 
  o_file.write("%s\t%s\t%s\t%s\t%s\n" % ("Quality",	"#1 ID",	"#2 ID",	"#1 String",	"#2 String"))
  for i in all_ids:
    l=last_25[i]
    if(len(l)==0):
      print("hi")
    indx = ids.index("Myntra_"+i)
    ll=clk_dict[i]
    rmv=[]
    for j in range(0,len(ll)):
      if "Myntra_"+ll[j] in ids:
        indxx=ids.index("Myntra_"+ll[j])
        o_file.write("%s\t%s\t%s\t%s\t%s\n" % ("1", i, ll[j], description[indx], description[indxx]))
        count+=1
        #print("1", i, ll[j], description[indx], description[indxx])
      else:
        rmv.append(ll[j])
    for j in range(0,15):
      indxx=ids.index("Myntra_"+l[j])
      o_file.write("%s\t%s\t%s\t%s\t%s\n" % ("0", i, l[j], description[indx], description[indxx]))
      count+=1
      #print("0", i, l[j], description[indx], description[indxx])
    for ii in range(0,len(rmv)):
      clk_dict[i].remove(rmv[ii])
 #   break
count

**Generate Test Data**

In [0]:
all_ids=[]
with open ("/content/drive/My Drive/Colab Notebooks/All Results.csv",'r') as f:
    csvreader=csv.reader(f, delimiter=',')
    next(f,None)
    for row in csvreader:
      all_ids.append(row[0])
all_ids=all_ids[12000:]
#all_ids=all_ids[:-1]

In [0]:
# top 100 articles using untrained BERT
with open("/content/drive/My Drive/Colab Notebooks/BERT_untrained_top_100_test.csv",'w') as o_file: 
  for w in all_ids:
    if "Myntra_"+w in model.wv:
      l = model.most_similar("Myntra_"+w,topn=150)
      i = ids.index("Myntra_"+w)
      article=description[i]
      query_article= article.split(",")[0]
      query_gender = article.split(",")[1]
      count=0
      string=""
      for item in l:
        i = ids.index(item[0])
        article=description[i]
        result_article = article.split(",")[0]
        result_gender = article.split(",")[1]
        if(query_article.strip()!=result_article.strip() or  query_gender.strip()!=result_gender.strip()):
          continue
        a=item[0]
        string +=str(a.split("_")[1])+","
        count+=1
        if count==100:
          break
      #print(string)
      o_file.write("%s\t%s\n" % ("Myntra_"+w, string))

In [0]:
top_hundred={}
with open("/content/drive/My Drive/Colab Notebooks/BERT_untrained_top_100_test.csv",'r') as d1:
    csvreader = csv.reader(d1, delimiter='\t')
   # next(d1, None)

    for row in csvreader:
      #print(row)
      id1=row[0].split("_")[1]
      rest=row[1][:-1].split(",")
      top_hundred[id1]=rest

In [0]:
with open("/content/drive/My Drive/Colab Notebooks/dataset/test.tsv",'w') as o_file: 
  o_file.write("%s\t%s\t%s\t%s\t%s\n" % ("Quality",	"#1 ID",	"#2 ID",	"#1 String",	"#2 String"))
  for i in all_ids:
    l=top_hundred[i]
    if(len(l)==0):
      print("hi")
    indx = ids.index("Myntra_"+i)
 #   count=0
    print(len(l))
    for j in range(0,len(l)):
      indxx=ids.index("Myntra_"+l[j])
      o_file.write("%s\t%s\t%s\t%s\t%s\n" % ("0", i, l[j], description[indx], description[indxx]))
      count+=1
